# 深度可分离卷积

卷积神经网络进化版，功能一致，但是速度更快、表示效率更高

卷积神经网络由卷积层和池化层组成，一般一个卷积层配一个池化层，多个卷积层配一个池化层也比较常见，局部相关，

一般应用场景：
- 2D卷积神经网络：图像识别
- 1D卷积神经网络：智能翻译

## 1. 导入包，并打印版本信息

In [ ]:
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

models = tf.keras.models
layers = tf.keras.layers
activations = tf.keras.activations
mnist = tf.keras.datasets.mnist
to_categorical = tf.keras.utils.to_categorical
optimizers = tf.keras.optimizers
losses = tf.keras.losses

print('tensorflow: ' + str(tf.__version__))
print('matplotlib: ' + str(matplotlib.__version__))
print('numpy: ' + str(np.__version__))

## 2. 准备样本数据

In [ ]:
(train_images, train_labels_original), (test_images, test_labels_original) = mnist.load_data()

train_images = train_images.reshape((len(train_images), 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((len(test_images), 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels_original)
test_labels = to_categorical(test_labels_original)

val_samples = 30000

data_train_images = train_images[val_samples:]
data_train_labels = train_labels[val_samples:]

val_images = train_images[:val_samples]
val_labels = train_labels[:val_samples]

## 3. 构建模型

In [ ]:
def build_model_1():
    model = models.Sequential()
    model.add(layers.SeparableConv2D(32, (3, 3), activation=activations.relu, input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.SeparableConv2D(64, (3, 3), activation=activations.relu))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation=activations.relu))
    model.add(layers.Dense(10, activation=activations.softmax))

    return model


def build_model_2():
    model = models.Sequential()
    model.add(layers.Conv2D(16, (5, 5), padding='same', input_shape=(28, 28, 1), activation=activations.relu))
    model.add(layers.MaxPool2D((2, 1)))
    model.add(layers.Conv2D(36, (5, 5), padding='same', activation=activations.relu))
    model.add(layers.MaxPool2D((2, 1)))
    model.add(layers.Dropout(0.25))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation=activations.relu))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation=activations.softmax))

    return model

model = build_model_1()

model.summary()

## 4. 编译模型

In [ ]:
model.compile(
    optimizer=optimizers.Adam(),
    loss=losses.categorical_crossentropy,
    metrics=['accuracy', 'mae', 'mse']
)

## 5. 训练模型

In [ ]:
history = model.fit(
    data_train_images,
    data_train_labels,
    epochs=10,
    batch_size=128,
    validation_data=(val_images, val_labels)
)

## 6. 评估模型

In [ ]:
model.evaluate(test_images, test_labels)

## 7. 预测结果

In [ ]:
x_samples = test_images[:100]
y_samples = test_labels_original[:100]
predications = model.predict(x_samples)

## 8. 使用图形显示100个结果

In [ ]:
plt.figure(figsize=(25, 25))
for i in range(100):
    plt.subplot(10, 10, i + 1)
    plt.xticks([])
    plt.yticks([])    
    plt.imshow(x_samples[i].reshape(28, 28), cmap=plt.cm.binary)
    plt.grid(True)
    if np.argmax(predications[i]) != test_labels_original[i]:       
        plt.xlabel('predict:' + str(np.argmax(predications[i])) + ', actual:' + str(y_samples[i]), color='red')
    else:
        plt.xlabel('predict:' + str(np.argmax(predications[i])) + ', actual:' + str(y_samples[i]))
plt.show()